In [1]:

%load_ext autoreload
%autoreload 2

import jax.numpy as jnp
import jax

from pbo.networks.learnable_q import TableQ


seed = 0
key = jax.random.PRNGKey(seed)
q_network_key, random_weights_key = jax.random.split(key)


q = TableQ(
    1,
    17,
    1,
    2,
    0.9,
    q_network_key,
    None,
    random_weights_key,
    None,
    zero_initializer=False,
)
batch_states = jnp.array([[15]])

q.n_actions_on_max

2

In [2]:
q.max_value(q.params, batch_states)

DeviceArray([[0.5195305]], dtype=float32)

In [3]:
print(q(q.params, batch_states[0], jnp.array([0])))
print(q(q.params, batch_states[0], jnp.array([1])))

[0.39729893]
[0.5195305]


In [4]:
q.n_actions_on_max

2

In [5]:
index_batch_states = jnp.arange(batch_states.shape[0])

indexes_states_mesh, indexes_actions_mesh = jnp.meshgrid(
    index_batch_states, q.index_actions_on_max, indexing="ij"
)
states = batch_states[indexes_states_mesh.flatten()]
actions = q.discrete_actions_on_max[indexes_actions_mesh.flatten()]

# Dangerous reshape: the indexing of meshgrid is 'ij'.
batch_values = q(q.params, states, actions).reshape((batch_states.shape[0], q.n_actions_on_max))

batch_values.max(axis=1).reshape((-1, 1))

DeviceArray([[0.5195305]], dtype=float32)